In [1]:
import time
class Timer:
    def __init__(self, s, iters):
        self.s = s
        self.iters = iters
    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, *args):
        print("\"" + self.s + "\"", "took", (time.perf_counter() - self.start)/self.iters*1000000, "microsec per db row")

In [2]:
import numpy as np
import pandas as pd

from starter import compute_fingerprint, load_database

with Timer("db loading", 1000000):
    fingerprints = load_database("database_fingerprints.npy")
    molecules = pd.read_csv("database.csv")
with open("query.txt") as q:
  query = q.readline()

"db loading" took 11.724437099997886 microsec per db row


In [3]:
k = 5

# Warmup
Just python!

In [4]:
def coverage(query, ref):
  query_f = compute_fingerprint(query)
  ref_f = compute_fingerprint(ref)
  return sum(query_f & ref_f) / sum(query_f)

with Timer("python coverage function", 1000):
    scores = [coverage(query, molecules["smiles"][i]) for i in range(1000)]
    topk = np.argsort(scores)[-k:]
print(scores[:k])
print(topk, [scores[i] for i in topk])

"python coverage function" took 3642.384099890478 microsec per db row
[0.2515592515592516, 0.22245322245322247, 0.2785862785862786, 0.22453222453222454, 0.30353430353430355]
[928  66  59 429 428] [0.4261954261954262, 0.4303534303534304, 0.4365904365904366, 0.46361746361746364, 0.498960498960499]


# Some sanity checks before we proceed
After we check that these scores are meaningful, we can proceed and just verify that our scores are exactly identical to this baseline.

In [5]:
print(coverage(molecules["smiles"][0], molecules["smiles"][0])) # should be 1
print(coverage(molecules["smiles"][0], molecules["smiles"][1])) # should be between 0 and 1
print(coverage("c1ccccc1", "C1CCCCC1")) # should be 0

1.0
0.746268656716418
0.0


# Speed things up with NumPy
We can get a massive speedup by using NumPy; everything is automatically vectorized.

In [6]:
with Timer("numpy direct bitwise_and", 100000):
    query_f = compute_fingerprint(query)
    scores = np.sum(query_f & fingerprints[:100000], axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numpy direct bitwise_and" took 58.05612300056964 microsec per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[94115 39023 94179 94185 38955] [0.60914761 0.59459459 0.6029106  0.6008316  0.5966736 ]


# Packing bits
Packing bits allows us to `&` 8 bits at a time.

Unpacking and summing is extremely inefficient though.

In [7]:
with Timer("numpy packed bits bitwise_and", 100000):
    query_f = compute_fingerprint(query)
    scores = np.sum(np.unpackbits(np.packbits(fingerprints[:100000], axis=1) & np.packbits(query_f)).reshape(-1, 2048), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numpy packed bits bitwise_and" took 12.280519999330863 microsec per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[94115 39023 94179 94185 38955] [0.60914761 0.59459459 0.6029106  0.6008316  0.5966736 ]


# Cython
We need C++ to get even faster; I want to use the native popcount instruction because [NumPy doesn't have a function to do that yet](https://github.com/numpy/numpy/issues/16325).

In [8]:
# Note: -O3 -march=native isn't just for fun, you really do need it to beat NumPy.
! which cython || conda install -y cython
! cython popcount.pyx
! gcc -Wall -O3 -g -lm -shared -pthread -fPIC -fwrapv -fno-strict-aliasing -mavx2 -fopt-info-vec-optimized -I$$CONDA_PREFIX/include/python3.6m -I$$CONDA_PREFIX/include -I$$CONDA_PREFIX/lib/python3.6/site-packages/numpy/core/include/ -o popcount.so popcount.c

/home/clive/anaconda3/envs/myenv/bin/cython
/home/clive/anaconda3/envs/myenv/lib/python3.6/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/clive/code/reverie-challenge/reverie-challenge/popcount.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /home/clive/anaconda3/envs/myenv/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /home/clive/anaconda3/envs/myenv/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/clive/anaconda3/envs/myenv/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from popcount.c:610:
/home/clive/anaconda3/envs/myenv/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPR

In [9]:
from popcount import inplace_popcount_32
with Timer("numpy packed bits popcounted", 1000000):
    query_f = compute_fingerprint(query)
    isct = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1) & np.packbits(query_f), dtype=np.uint32)
    inplace_popcount_32(isct)
    scores = np.sum(isct.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numpy packed bits popcounted" took 1.7242891000350937 microsec per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


Now that we're processing more than 5x faster than it takes to read the database off the SSD, we can declare victory, but also I'm curious how far we can go.

# Fusing operations
This is sort of a diminishing returns scenario; I needed `-O3 -march=native` to run the bitwise and faster than NumPy.

In [10]:
from popcount import fused_popcount_bitwise_and
with Timer("numpy fused bitwise_and + popcount", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint32)
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1), dtype=np.uint32)
    fused_popcount_bitwise_and(query_packed, fingerprints_packed)
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numpy fused bitwise_and + popcount" took 1.1449584000511095 microsec per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


# Fusing the packing too
The packbits function is now the slowest part, at about 80% of the runtime. What if we don't do that at all?

It ends up being slower than NumPy because NumPy is [using proper SIMD instructions for this](https://github.com/numpy/numpy/blob/97d2db483fc0ffd46f38d0e1c39d5fc001e33197/numpy/core/src/multiarray/compiled_base.c#L1543). This is okay, we now have the tools to do this properly.

In [11]:
from popcount import fused_popcount_bitwise_and_notpacked_count
with Timer("fused notpacked popcount", 1000000):
    query_f = compute_fingerprint(query).astype(np.uint8)
    fused_popcount_bitwise_and_notpacked_count(query_f, fingerprints[:1000000])
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"fused notpacked popcount" took 2.180406400002539 microsec per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


# Endgame: AVX2
AVX2 is supported by most modern processors, including the Intel processor that the specified MBP 2019 has.

We get an absurdly high throughput: we're processing about 5 boolean database entries per nanosecond.

In [12]:
from popcount import fused_popcount_avx2
with Timer("avx2", 1000000):
    query_f = compute_fingerprint(query).astype(np.uint8)
    scores = fused_popcount_avx2(query_f, fingerprints[:1000000]) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"avx2" took 0.22520780004560947 microsec per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


# Comments and things to improve

Algorithmic complexity is O(database size), since it must iterate over all the booleans in every row. I'm fairly certain that this cannot be asymptotically improved - for example in the worst case every molecule in the database contains all substructures (all 2048 bits are true).

Memory usage is high; we're reading the entire database into memory and keeping it there. There isn't really any way around keeping O(database size) in memory because SSD bandwidth is far too low to support streaming from disk, but an 8x memory usage/bandwidth bottleneck improvement can probably be found by bitpacking the database beforehand.

# Bonus: Overtime stuff

This was done for fun significantly over the 6hr time limit; please don't count this if you're evaluating quantity of code produced.

I did no memory-related optimizations mostly because I was more interested in playing with AVX, and because up until AVX the whole thing was compute-bottlenecked rather than memory-bottlenecked. However it's trivial to do so; just don't count the bitpacking step from the `Fusing operations` section, and you get (only!) a slight speedup over the AVX implementation as seen below.

In [13]:
from popcount import fused_popcount_bitwise_and
with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1), dtype=np.uint32)
with Timer("pre-packed rows", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint32)
    fused_popcount_bitwise_and(query_packed, fingerprints_packed)
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"pre-packing process" took 0.6740187000250444 microsec per db row
"pre-packed rows" took 0.17853230005130172 microsec per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[329123 324807 324852 329172 324758] [0.66735967 0.66735967 0.66943867 0.66735967 0.66735967]


Unfinished attempt at using popcount64 directly on 64-bit ints, instead of looping `__builtin_popcount` on 32-bit ints.

In [14]:
from popcount import fused_popcount64_bitwise_and
with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1, bitorder='little'), dtype=np.uint64)
with Timer("pre-packed rows", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f, bitorder='little'), dtype=np.uint64)
    fused_popcount64_bitwise_and(query_packed, fingerprints_packed)
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"pre-packing process" took 0.6325727000366896 microsec per db row
"pre-packed rows" took 0.1641679999884218 microsec per db row
[2.44224911e+16 9.03778830e+15 2.53200250e+16 1.19146760e+16
 5.83236742e+15]
[ 24859 249809  98695 313458 310658] [3.83504309e+16 3.83507062e+16 3.83503865e+16 3.83503088e+16
 3.83501115e+16]


Also not working correctly, but AVX2 emulated popcnt [can be 30% faster](https://stackoverflow.com/a/50082218) than the dedicated instruction.

In [15]:
from popcount import fused_avx2_emulated_popcount
with Timer("pre-packing process", 1000000):
    fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1), dtype=np.uint64)
with Timer("pre-packed rows", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint64)
    fused_avx2_emulated_popcount(query_packed, fingerprints_packed)
    scores = np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"pre-packing process" took 0.7992424999829382 microsec per db row
"pre-packed rows" took 0.16936449997592717 microsec per db row
[4.03833063e+15 2.62815452e+16 3.77945367e+15 3.34960946e+16
 8.88579474e+15]
[439306  28215  93292 489702 106223] [3.83505627e+16 3.83506231e+16 3.83505872e+16 3.83507498e+16
 3.83504301e+16]


Some additional cleverness that I do not have time to explore:
- A slight (2x?) additional memory usage improvement could be made by `blosc`-compressing the bitpacked database in-memory, though this is unlikely to change the overall runtime.
- AVX-512 would be fun but I don't have ice lake hardware sadly
- OpenCL was originally planned but no time, and also I think my laptop iGPU doesn't support it anyway.

# Bonus: Threading
Because why not? The AVX2 code is mostly in C-land away from the GIL so we might be able to get close to linear speedup in the number of cores (regular cores; hyperthreading probably does not help for this because AVX2 resources are shared).

In practice this doesn't actually help at all, probably because the AVX2 code is so fast that there are other bottlenecks e.g. system memory bandwidth. 2GB dataset in 0.2s is 10GBps, which is somewhat over half the theoretical memory bandwidth my machine should have.

This also points toward there being very little point in using OpenCL: unless you have a discrete GPU with dedicated high bandwidth video memory, which isn't the case on MBP 2019, it's not going to get you much further, and more likely will slow things down.

In [16]:
import threading
from popcount import fused_popcount_bitwise_and

nthreads = 4
topk_per_thread = [None] * nthreads
topk_per_thread_scores = [None] * nthreads

def thread_func(query_f, threadid):
    start = threadid*len(fingerprints)//nthreads
    end = (threadid+1)*len(fingerprints)//nthreads # Assumes that nranks divides len(fingerprints)
    scores = fused_popcount_avx2(query_f, fingerprints[start:end])

    topk_per_thread[threadid] = np.argpartition(-scores, k)[:k]
    topk_per_thread_scores[threadid] = scores[topk_per_thread[threadid]] / np.sum(query_f)
    topk_per_thread[threadid] += start

with Timer("threaded avx2", 1000000):
    query_f = compute_fingerprint(query)
    query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint32)
    threads = [None] * nthreads
    for i in range(nthreads):
        threads[i] = threading.Thread(target=thread_func, args=(query_f, i))
        threads[i].start()
    for i in range(nthreads):
        threads[i].join()
    topk_arg = np.argpartition(-np.concatenate(topk_per_thread_scores), k)[:k]
    scores = np.concatenate(topk_per_thread_scores)[topk_arg]
    topk = np.concatenate(topk_per_thread)[topk_arg]
print(topk, scores)

"threaded avx2" took 0.35625459998846054 microsec per db row
[324807 324852 324758 329172 329123] [0.66735967 0.66943867 0.66735967 0.66735967 0.66735967]


# Bonus: A failed attempt at using Numba
Cython has some ... interoperability issues ... with Numba, so using the popcount kernel isn't possible.

Using purely Numba (without the popcount kernel) is way slower.

In [17]:
try:
    from numba import jit
except ImportError:
    ! conda install -y numba
    from numba import jit

In [18]:
import numba
# from numba.extending import get_cython_function_address
# import numpy.ctypeslib as npct
# import ctypes

# array_1d_uint32 = npct.ndpointer(dtype=np.uint32, ndim=1, flags='CONTIGUOUS')
# addr = get_cython_function_address("popcount", "_fused_popcount_bitwise_and")
# functype = ctypes.CFUNCTYPE(None, array_1d_uint32, array_1d_uint32)
# fused_popcount_bitwise_and = functype(addr)

@numba.jit(nopython=True)
def numba_func(query_f, fingerprints):
    return np.sum(query_f & fingerprints[:100000], axis=1) / np.sum(query_f)

# @numba.jit(nopython=True)
# def numba_func(query_f, fingerprints):
#     query_packed = np.frombuffer(np.packbits(query_f), dtype=np.uint32)
#     fingerprints_packed = np.frombuffer(np.packbits(fingerprints[:1000000], axis=1), dtype=np.uint32)
#     fused_popcount_bitwise_and(query_packed, fingerprints_packed)
#     return np.sum(fingerprints_packed.reshape(-1, 2048//32), axis=1) / np.sum(query_f)

with Timer("numba", 100000):
    query_f = compute_fingerprint(query)
    scores = numba_func(query_f, fingerprints)
    topk = np.argpartition(-scores, k)[:k]
print(scores[:k])
print(topk, scores[topk])

"numba" took 105.78900799970143 microsec per db row
[0.25155925 0.22245322 0.27858628 0.22453222 0.3035343 ]
[94115 39023 94179 94185 38955] [0.60914761 0.59459459 0.6029106  0.6008316  0.5966736 ]
